# UnPythinic vs Pythonic __dict__:

In [5]:
 class Vector:
    
    def __init__(self, **coords):
        private_coords = {'_'+ k: v for k,v in coords.items()}
        self.__dict__.update(private_coords)
        
    def __getattr__(self, name):
        private_name = '_'+name
        try:
            return self.__dict__[private_name]
        except keyError:
            return '{} - Attribute not availabe in {}'.format(name, self.__class__.__name__) 
        print('Name = {}'.format(getattr(self,private_name)))
    
    def __setattr__(self, name, value):
        raise AttributeError("Don't set attribute {}".format(name))
        
    def __delattr__(self, name):
        raise AttributeError("Don't delete attribute {}".format(name))
        
    def __repr__(self):
        return '{} - ({})'.format(
        self.__class__.__name__, ','.join('{k}={v}'.format(
            k = k[1:],
            v = self.__dict__[k])
            for k in sorted(self.__dict__.keys())))


In [6]:
 class ColoredVector(Vector):
    
    COLOR_INDEX = ('red', 'green', 'blue')
    
    def __init__(self, red, green , blue, **coords):
        super().__init__(**coords)
        self.__dict__['color'] = [red, green, blue]
        
    def __getattr__(self, name):
        try:
            channel = ColoredVector.COLOR_INDEX.index(name)
        except ValueError:
            return super().__getattr__(name)
        else:
            return self.__dict__['color'][channel]
            
    def __setattr__(self, name, value):
        try:
            channel = ColoredVector.COLOR_INDEX.index(name)
        except ValueError:
            super().__setattr__(name, value)
        else:
            self.__dict__['color'][channel] = value
            
    def __repr__(self):
        keys = set((self.__dict__.keys()))
        keys.discard('color')
        coords = ','.join('{k}: {v}'.format(
            k = k[1:],
            v = self.__dict__[k])
            for k in sorted(keys))
        return '{cls} ({red}, {green}, {blue}, {coords})'.format(
            cls = self.__class__.__name__,
            red = self.red,
            green = self.green,
            blue = self.blue,
            coords = coords
        )

In [7]:
cv = ColoredVector(red =10, green = 20, blue = 30, p=23, q =23)

In [9]:
# Pythonic
vars(cv)

{'_p': 23, '_q': 23, 'color': [10, 20, 30]}

In [13]:
# UnPythonic
cv.__dict__

{'_p': 23, '_q': 23, 'color': [10, 20, 30]}

In [15]:
# Pythonic
len(vars(cv)['color'])

3

In [16]:
# UnPythonic
cv.__dict__['color'].__len__()

3

In [17]:
# Pythonic:
vars(cv)['color'] = [23,23,23]

In [18]:
# UnPythonic:
cv.__dict__['color'] = [23,32,32]

# getattribute:

In [23]:
class LoggingProxy:
    
    def __init__(self, target):
        super().__setattr__('target', target)
        
    def __getattribute__(self, name):
        target = super().__getattribute__('target')
        
        try:
            value = getattr(target, name)
        except AttributeError as e:
            raise AttributeError('{} could not forward {} to {}'.format(
                super().__getattribute__('__class__').__name__,
                name,
                target
            ))
        print('Retrived Attribute {} = {} from {}'.format(name, value, target))
        return value

In [24]:
cv = ColoredVector(red =10, green = 20, blue = 30, p=23, q =23)


In [25]:
cw = LoggingProxy(cv)

In [26]:
cw.p

Retrived Attribute p = 23 from ColoredVector (10, 20, 30, p: 23,q: 23)


23

In [27]:
cw.q

Retrived Attribute q = 23 from ColoredVector (10, 20, 30, p: 23,q: 23)


23

In [28]:
cw.green

Retrived Attribute green = 20 from ColoredVector (10, 20, 30, p: 23,q: 23)


20

In [39]:
class LoggingProxy:
    
    def __init__(self, target):
        super().__setattr__('target', target)
        
    def __getattribute__(self, name):
        target = super().__getattribute__('target')
        
        try:
            value = getattr(target, name)
        except AttributeError as e:
            raise AttributeError('{} could not forward {} to {}'.format(
                super().__getattribute__('__class__').__name__,
                name,
                target
            ))
        print('Retrived Attribute {} = {} from {}'.format(name, value, target))
        return value
    
    def __setattr__(self, name, value):
        target = super().__getattribute__('target')
        
        try:
            setattr(target, name, value)
        except AttributeError as e:
            raise AttributeError('{} could not forward {} to {}'.format(
                super().__getattribute__('__class__').__name__,
                name,
                target
            ))
        print('Set Attribute {} = {} from {}'.format(name, value, target))

In [40]:
cv = ColoredVector(red =10, green = 20, blue = 30, p=23, q =23)


In [41]:
cw = LoggingProxy(cv)

Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (10, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (10, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (10, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (10, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (10, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (10, 20, 30, p: 23,q: 23)


In [42]:
cw.red =23

Set Attribute red = 23 from ColoredVector (23, 20, 30, p: 23,q: 23)


In [43]:
cw.red

Retrived Attribute red = 23 from ColoredVector (23, 20, 30, p: 23,q: 23)


23

In [44]:
cw.p =2332

AttributeError: LoggingProxy could not forward p to ColoredVector (23, 20, 30, p: 23,q: 23)

# Attribute Special Methods:

In [45]:
cw

Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (23, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (23, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (23, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (23, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (23, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (23, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (23, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (23, 20, 30, p: 23,q: 23)
Retrived Attribute __class__ = <class '__main__.ColoredVector'> from ColoredVector (23, 20, 30, p: 23,q: 23)
Retrived Attribute 

In [49]:
cw.__repr__()

Retrived Attribute __repr__ = <bound method ColoredVector.__repr__ of ColoredVector (23, 20, 30, p: 23,q: 23)> from ColoredVector (23, 20, 30, p: 23,q: 23)


'ColoredVector (23, 20, 30, p: 23,q: 23)'

In [47]:
repr(cw)

'<__main__.LoggingProxy object at 0x000000764B5DC438>'

In [54]:
class LoggingProxy:
    
    def __init__(self, target):
        super().__setattr__('target', target)
        
    def __getattribute__(self, name):
        target = super().__getattribute__('target')
        
        try:
            value = getattr(target, name)
        except AttributeError as e:
            raise AttributeError('{} could not forward {} to {}'.format(
                super().__getattribute__('__class__').__name__,
                name,
                target
            ))
        print('Retrived Attribute {} = {} from {}'.format(name, value, target))
        return value
    
    def __setattr__(self, name, value):
        target = super().__getattribute__('target')
        
        try:
            setattr(target, name, value)
        except AttributeError as e:
            raise AttributeError('{} could not forward {} to {}'.format(
                super().__getattribute__('__class__').__name__,
                name,
                target
            ))
        print('Set Attribute {} = {} from {}'.format(name, value, target))
        
    def repr(self):
        target = super().__getattribute__('target')
        repr_callable = getattr(target, '__repr__')
        return repr_callable()

In [55]:
cv = ColoredVector(red =10, green = 20, blue = 30, p=23, q =23)
cw = LoggingProxy(cv)

In [56]:
repr(cv)

'ColoredVector (10, 20, 30, p: 23,q: 23)'

In [57]:
cv.__repr__()

'ColoredVector (10, 20, 30, p: 23,q: 23)'

# Where are the Methods:

In [58]:
v = Vector(x =2, y=23)

In [59]:
v.__dict__

{'_x': 2, '_y': 23}

In [60]:
v.__class__

__main__.Vector

In [61]:
v.__class__.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Vector.__init__(self, **coords)>,
              '__getattr__': <function __main__.Vector.__getattr__(self, name)>,
              '__setattr__': <function __main__.Vector.__setattr__(self, name, value)>,
              '__delattr__': <function __main__.Vector.__delattr__(self, name)>,
              '__repr__': <function __main__.Vector.__repr__(self)>,
              '__dict__': <attribute '__dict__' of 'Vector' objects>,
              '__weakref__': <attribute '__weakref__' of 'Vector' objects>,
              '__doc__': None})

In [63]:
v.__class__.__dict__['__repr__'](v)

'Vector - (x=2,y=23)'

In [64]:
v.__class__.__dict__['name'] = 'optimus'

TypeError: 'mappingproxy' object does not support item assignment

In [65]:
setattr(v.__class__, 'name', 'optimus')

In [66]:
Vector.name

'optimus'

# slot:

In [67]:
import sys
d = {}
sys.getsizeof(d)

240

In [68]:
class Resistor:
    
    def __init__(self, resistance_ohm, tolerance_percent, power_watts):
        self.resistance_ohm = resistance_ohm
        self.tolerance_percent = tolerance_percent
        self.power_watts = power_watts
        
    

In [76]:
r = Resistor(10, 5,8.25)

In [77]:
sys.getsizeof(r)+sys.getsizeof(r.__dict__)

168

In [80]:
r.cost_dollers = 203

In [81]:
sys.getsizeof(r)+sys.getsizeof(r.__dict__)

168

In [85]:
sys.getsizeof(r.__dict__)

112

In [86]:
class Resistor:
    
    __slots__ = ['resistance_ohm', 'tolerance_percent', 'power_watts']
    
    def __init__(self, resistance_ohm, tolerance_percent, power_watts):
        self.resistance_ohm = resistance_ohm
        self.tolerance_percent = tolerance_percent
        self.power_watts = power_watts
        
    

In [87]:
r = Resistor(10, 5,8.25)

In [88]:
r.resistance_ohm

10

In [89]:
r.cost =32

AttributeError: 'Resistor' object has no attribute 'cost'

In [90]:
r.__dict__

AttributeError: 'Resistor' object has no attribute '__dict__'

In [91]:
r.__repr__

<method-wrapper '__repr__' of Resistor object at 0x000000764B5FA688>

In [92]:
r.__class__

__main__.Resistor

In [93]:
r.tolerance_percent =23

In [94]:
r.name =23

AttributeError: 'Resistor' object has no attribute 'name'